In [14]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:05:08)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

## lesson 3.1

In [21]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row

edgeList = [(1, 2), (1, 3), (1, 4), (2, 3), (2, 5), (3, 4), (3, 5), (3, 6), (3, 7)]

graphData = sparkSession.sparkContext.parallelize(edgeList).map(lambda (src, dst): Row(src, dst))

graphSchemaAB = StructType([StructField('A', IntegerType(), nullable=False),
                       StructField('B', StringType(), nullable=False)])

abDF = sparkSession.createDataFrame(graphData, graphSchemaAB)

graphSchemaBC1 = StructType([StructField('B', IntegerType(), nullable=False),
                       StructField('C', StringType(), nullable=False)])
bcDF = sparkSession.createDataFrame(graphData, graphSchemaBC1)

abDF.show()

+---+---+
|  A|  B|
+---+---+
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  3|
|  2|  5|
|  3|  4|
|  3|  5|
|  3|  6|
|  3|  7|
+---+---+



In [16]:
abDF.show()

+---+---+
|  A|  B|
+---+---+
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  3|
|  2|  5|
|  3|  4|
|  3|  5|
|  3|  6|
|  3|  7|
+---+---+



In [17]:
bcDF.show()

+---+---+
|  B|  C|
+---+---+
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  3|
|  2|  5|
|  3|  4|
|  3|  5|
|  3|  6|
|  3|  7|
+---+---+



In [18]:
abcDF = abDF.join(bcDF, "B")
abcDF.show()

+---+---+---+
|  B|  A|  C|
+---+---+---+
|  3|  1|  4|
|  3|  1|  5|
|  3|  1|  6|
|  3|  1|  7|
|  3|  2|  4|
|  3|  2|  5|
|  3|  2|  6|
|  3|  2|  7|
|  2|  1|  3|
|  2|  1|  5|
+---+---+---+



In [19]:
abcDF = abDF.join(bcDF, "B")
abcDF.filter("A = 1").show()


+---+---+---+
|  B|  A|  C|
+---+---+---+
|  3|  1|  4|
|  3|  1|  5|
|  3|  1|  6|
|  3|  1|  7|
|  2|  1|  3|
|  2|  1|  5|
+---+---+---+



In [20]:
abcDF.drop("B") \
    .groupBy("A", "C") \
    .count() \
    .filter("A = 1") \
    .show()

+---+---+-----+
|  A|  C|count|
+---+---+-----+
|  1|  6|    1|
|  1|  7|    1|
|  1|  3|    1|
|  1|  4|    1|
|  1|  5|    2|
+---+---+-----+



## lesson 3.2

In [22]:
from graphframes import * 

vertices = sparkSession.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = sparkSession.createDataFrame([
  ("1", "2", "friend"),
  ("2", "1", "friend"),
  ("1", "3", "friend"),
  ("3", "1", "friend"),
  ("1", "4", "friend"),
  ("4", "1", "friend"),
  ("2", "3", "friend"), 
  ("3", "2", "friend"),
  ("2", "5", "friend"),
  ("5", "2", "friend"),
  ("3", "4", "friend"),
  ("4", "3", "friend"),
  ("3", "5", "friend"),
  ("5", "3", "friend"),
  ("3", "6", "friend"),
  ("6", "3", "friend"),
  ("3", "7", "friend"),
  ("7", "3", "friend")
], ["src", "dst", "relationship"])

g = GraphFrame(vertices, edges)


In [23]:
g.vertices.filter("age > 30")

DataFrame[id: string, name: string, age: bigint, gender: string, university: string]

In [24]:
g.inDegrees.filter("inDegree > 2").show()

+---+--------+
| id|inDegree|
+---+--------+
|  3|       6|
|  1|       3|
|  2|       3|
+---+--------+



In [25]:
g.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
|  7|       1|
|  3|       6|
|  5|       2|
|  6|       1|
|  1|       3|
|  4|       2|
|  2|       3|
+---+--------+



In [26]:
g.find("(a)-[e]->(b); (b)-[e2]->(a)").show(15)

+--------------------+------------+--------------------+------------+
|                   a|           e|                   b|          e2|
+--------------------+------------+--------------------+------------+
|  [1,Alex,28,M,MIPT]|[1,4,friend]| [4,Pavel,30,M,MIPT]|[4,1,friend]|
| [4,Pavel,30,M,MIPT]|[4,1,friend]|  [1,Alex,28,M,MIPT]|[1,4,friend]|
|[3,Natasha,27,F,S...|[3,2,friend]| [2,Emeli,28,F,MIPT]|[2,3,friend]|
| [2,Emeli,28,F,MIPT]|[2,1,friend]|  [1,Alex,28,M,MIPT]|[1,2,friend]|
| [2,Emeli,28,F,MIPT]|[2,5,friend]|  [5,Oleg,35,M,MIPT]|[5,2,friend]|
|[3,Natasha,27,F,S...|[3,5,friend]|  [5,Oleg,35,M,MIPT]|[5,3,friend]|
|  [1,Alex,28,M,MIPT]|[1,3,friend]|[3,Natasha,27,F,S...|[3,1,friend]|
|[3,Natasha,27,F,S...|[3,1,friend]|  [1,Alex,28,M,MIPT]|[1,3,friend]|
|  [5,Oleg,35,M,MIPT]|[5,3,friend]|[3,Natasha,27,F,S...|[3,5,friend]|
| [2,Emeli,28,F,MIPT]|[2,3,friend]|[3,Natasha,27,F,S...|[3,2,friend]|
|[3,Natasha,27,F,S...|[3,7,friend]|   [7,Ilya,29,M,MSU]|[7,3,friend]|
| [4,Pavel,30,M,MIPT

In [27]:
motifs = g.find("(A)-[]->(B); (B)-[]->(C)")
motifs.show(15)

+--------------------+--------------------+--------------------+
|                   A|                   B|                   C|
+--------------------+--------------------+--------------------+
|   [7,Ilya,29,M,MSU]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|   [6,Ivan,30,M,MSU]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
| [4,Pavel,30,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|[3,Natasha,27,F,S...|
|[3,Natasha,27,F,S...|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...|
| [2,Emeli,28,F,MIPT]|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...|
|[3,Natasha,27,F,S...|   [6,Ivan,30,M,MSU]|[3,Natasha,27,F,S...|
|[3,Natasha,27,F,S...|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|
| [4,Pavel,30,M,MIPT]|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|
| [2,Emeli,28,F,MIPT]|  [

In [28]:
motifs = g.find("(A)-[]->(B); (B)-[]->(C)").filter("A.id != C.id")
motifs.show(15)

+-------------------+--------------------+--------------------+
|                  A|                   B|                   C|
+-------------------+--------------------+--------------------+
| [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|  [6,Ivan,30,M,MSU]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
| [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|[4,Pavel,30,M,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|[2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|   [7,Ilya,29,M,MSU]|
|[2,Emeli,28,F,MIPT]|  [5,Oleg,35,M,MIPT]|[3,Natasha,27,F,S...|
|[4,Pavel,30,M,MIPT]|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|
|[2,Emeli,28,F,MIPT]|  [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...|
| [1,Alex,28,M,MIPT]| [4,Pavel,30,M,MIPT]|[3,Natasha,27,F,S...|
| [5,Oleg,35,M,MIPT]| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|
| [1,Alex,28,M,MIPT]| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|
|  [7,Ilya,29,M,MSU]|[3,Natasha,27,F,S...|  [5,Oleg,35,M,MIPT]|
|  [6,Ivan,30,M,MSU]|[3,Natasha,27,F,S..

In [30]:
AC = motifs.selectExpr("A.id as A", "C.id as C")
AC.show(15)

+---+---+
|  A|  C|
+---+---+
|  5|  7|
|  6|  7|
|  1|  7|
|  4|  7|
|  2|  7|
|  2|  3|
|  4|  3|
|  2|  3|
|  1|  3|
|  5|  3|
|  1|  3|
|  7|  5|
|  6|  5|
|  1|  5|
|  4|  5|
+---+---+
only showing top 15 rows



In [31]:
AC.groupBy("A", "C").count().filter("A = 1").show()

+---+---+-----+
|  A|  C|count|
+---+---+-----+
|  1|  4|    1|
|  1|  2|    1|
|  1|  5|    2|
|  1|  3|    2|
|  1|  7|    1|
|  1|  6|    1|
+---+---+-----+



In [32]:
AC.groupBy("A", "C").count().show(15)

+---+---+-----+
|  A|  C|count|
+---+---+-----+
|  7|  1|    1|
|  4|  1|    1|
|  4|  2|    2|
|  5|  4|    1|
|  1|  4|    1|
|  4|  6|    1|
|  5|  1|    2|
|  2|  3|    2|
|  7|  6|    1|
|  5|  7|    1|
|  1|  2|    1|
|  5|  2|    1|
|  2|  6|    1|
|  2|  4|    2|
|  5|  3|    1|
+---+---+-----+
only showing top 15 rows



## lesson 3.3

In [34]:
results = g.triangleCount()
results.show()

+-----+---+-------+---+------+----------+
|count| id|   name|age|gender|university|
+-----+---+-------+---+------+----------+
|    0|  7|   Ilya| 29|     M|       MSU|
|    3|  3|Natasha| 27|     F|     SPbSU|
|    1|  5|   Oleg| 35|     M|      MIPT|
|    0|  6|   Ivan| 30|     M|       MSU|
|    2|  1|   Alex| 28|     M|      MIPT|
|    1|  4|  Pavel| 30|     M|      MIPT|
|    2|  2|  Emeli| 28|     F|      MIPT|
+-----+---+-------+---+------+----------+



In [35]:
edgeList = [(1, 2), (1, 3), (1, 4), (2, 3), (2, 5), (3, 4), (3, 5), (3, 6), (3, 7)]

In [36]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row

edgeList = [(1, 2), (1, 3), (1, 4), (2, 3), (2, 5), (3, 4), (3, 5), (3, 6), (3, 7)]

graphData = sparkSession.sparkContext.parallelize(edgeList).map(lambda (src, dst): Row(src, dst))

graphSchemaAB = StructType([StructField('A', IntegerType(), nullable=False),
                       StructField('B', StringType(), nullable=False)])

abDF = sparkSession.createDataFrame(graphData, graphSchemaAB)

abDF.show()

+---+---+
|  A|  B|
+---+---+
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  3|
|  2|  5|
|  3|  4|
|  3|  5|
|  3|  6|
|  3|  7|
+---+---+



In [37]:
graphSchemaBC1 = StructType([StructField('B', IntegerType(), nullable=False),
                       StructField('C1', StringType(), nullable=False)])
bc1DF = sparkSession.createDataFrame(graphData, graphSchemaBC1)

graphSchemaAC2 = StructType([StructField('A', IntegerType(), nullable=False),
                       StructField('C2', StringType(), nullable=False)])
ac2DF = sparkSession.createDataFrame(graphData, graphSchemaAC2)

In [38]:
bc1DF.show()

+---+---+
|  B| C1|
+---+---+
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  3|
|  2|  5|
|  3|  4|
|  3|  5|
|  3|  6|
|  3|  7|
+---+---+



In [39]:
ac2DF.show()

+---+---+
|  A| C2|
+---+---+
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  3|
|  2|  5|
|  3|  4|
|  3|  5|
|  3|  6|
|  3|  7|
+---+---+



In [40]:
abc1DF = abDF.join(bc1DF, "B")
abc1DF.show()

+---+---+---+
|  B|  A| C1|
+---+---+---+
|  3|  1|  4|
|  3|  1|  5|
|  3|  1|  6|
|  3|  1|  7|
|  3|  2|  4|
|  3|  2|  5|
|  3|  2|  6|
|  3|  2|  7|
|  2|  1|  3|
|  2|  1|  5|
+---+---+---+



In [41]:
abc1c2DF = abc1DF.join(ac2DF, "A")
abc1c2DF.show()

+---+---+---+---+
|  A|  B| C1| C2|
+---+---+---+---+
|  1|  3|  4|  2|
|  1|  3|  4|  3|
|  1|  3|  4|  4|
|  1|  3|  5|  2|
|  1|  3|  5|  3|
|  1|  3|  5|  4|
|  1|  3|  6|  2|
|  1|  3|  6|  3|
|  1|  3|  6|  4|
|  1|  3|  7|  2|
|  1|  3|  7|  3|
|  1|  3|  7|  4|
|  1|  2|  3|  2|
|  1|  2|  3|  3|
|  1|  2|  3|  4|
|  1|  2|  5|  2|
|  1|  2|  5|  3|
|  1|  2|  5|  4|
|  2|  3|  4|  3|
|  2|  3|  4|  5|
+---+---+---+---+
only showing top 20 rows



In [42]:
abc1c2DF.filter("C1 = C2").show()

+---+---+---+---+
|  A|  B| C1| C2|
+---+---+---+---+
|  1|  3|  4|  4|
|  1|  2|  3|  3|
|  2|  3|  5|  5|
+---+---+---+---+



In [43]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [44]:
{
    abc1c2DF.filter("C1 = C2")
    .select(array(col("A"), col("B"), col("C1")).alias("triangleVertices"))
    .select(explode("triangleVertices").alias("triangleVertex"))
    .groupBy("triangleVertex")
    .count()
    .show()
}

+--------------+-----+
|triangleVertex|count|
+--------------+-----+
|             3|    3|
|             5|    1|
|             1|    2|
|             4|    1|
|             2|    2|
+--------------+-----+



{None}

In [46]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row

edgeList = [(1, 2), (1, 3), (1, 4), (2, 3), (2, 5), (3, 4), (3, 5), (3, 6), (3, 7)]


graphData = sparkSession.sparkContext.parallelize(edgeList).map(lambda (src, dst): Row(src, dst))

graphSchemaAB = StructType([StructField('A', IntegerType(), nullable=False),
                       StructField('B', StringType(), nullable=False)])

abDF = sparkSession.createDataFrame(graphData, graphSchemaAB)

graphSchemaBC1 = StructType([StructField('B', IntegerType(), nullable=False),
                       StructField('C1', StringType(), nullable=False)])
bc1DF = sparkSession.createDataFrame(graphData, graphSchemaBC1)

graphSchemaAC2 = StructType([StructField('A', IntegerType(), nullable=False),
                       StructField('C2', StringType(), nullable=False)])
ac2DF = sparkSession.createDataFrame(graphData, graphSchemaAC2)

abc1DF = abDF.join(bc1DF, "B")
abc1c2DF = abc1DF.join(ac2DF, "A")


vertexTriangle = abc1c2DF.filter("C1 = C2") \
    .select(array(col("A"), col("B"), col("C1")).alias("triangleVertices")) \
    .select(explode("triangleVertices").alias("triangleVertex")) \
    .groupBy("triangleVertex") \
    .count() \
    .show()
    

+--------------+-----+
|triangleVertex|count|
+--------------+-----+
|             3|    3|
|             5|    1|
|             1|    2|
|             4|    1|
|             2|    2|
+--------------+-----+



In [47]:
edgeListBC = sparkSession.sparkContext.broadcast(set(edgeList))

# Define udf
from pyspark.sql.functions import udf

def isInEdgeList(src, dst):
    return (int(src), int(dst)) in edgeListBC.value

udfIsInEdgeList = udf(isInEdgeList, StringType())

abc1DF.withColumn("isTriangle", udfIsInEdgeList("A", "C1")).show()

+---+---+---+----------+
|  B|  A| C1|isTriangle|
+---+---+---+----------+
|  3|  1|  4|      true|
|  3|  1|  5|     false|
|  3|  1|  6|     false|
|  3|  1|  7|     false|
|  3|  2|  4|     false|
|  3|  2|  5|      true|
|  3|  2|  6|     false|
|  3|  2|  7|     false|
|  2|  1|  3|      true|
|  2|  1|  5|     false|
+---+---+---+----------+



In [48]:
from graphframes import * 

vertices = sparkSession.createDataFrame([
  ("1", "Alex", 28, "M", "MIPT"),
  ("2", "Emeli", 28, "F", "MIPT"),
  ("3", "Natasha", 27, "F", "SPbSU"),
  ("4", "Pavel", 30, "M", "MIPT"),
  ("5", "Oleg", 35, "M", "MIPT"),
  ("6", "Ivan", 30, "M", "MSU"),
  ("7", "Ilya", 29, "M", "MSU")], ["id", "name", "age", "gender", "university"])

edges = sparkSession.createDataFrame([
  ("1", "2", "friend"),
  ("1", "3", "friend"),
  ("1", "4", "friend"),
  ("2", "3", "friend"),
  ("2", "5", "friend"),
  ("3", "4", "friend"),
  ("3", "5", "friend"),
  ("3", "6", "friend"),
  ("3", "7", "friend")
], ["src", "dst", "type"])

g = GraphFrame(vertices, edges)

In [49]:
triangles = g.find("(A)-[]->(B); (B)-[]->(C); (A)-[]->(C)")

In [50]:
triangles.show()

+-------------------+--------------------+--------------------+
|                  A|                   B|                   C|
+-------------------+--------------------+--------------------+
| [1,Alex,28,M,MIPT]|[3,Natasha,27,F,S...| [4,Pavel,30,M,MIPT]|
| [1,Alex,28,M,MIPT]| [2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|
|[2,Emeli,28,F,MIPT]|[3,Natasha,27,F,S...|  [5,Oleg,35,M,MIPT]|
+-------------------+--------------------+--------------------+



In [51]:
vertexTriangles = triangles.selectExpr("A.id as A", "B.id as B", "C.id as C") \
    .select(array(col("A"), col("B"), col("C")).alias("triangleVertices")) \
    .select(explode("triangleVertices").alias("id")) \
    .groupBy("id") \
    .count()

In [52]:
vertexTriangles.show()

+---+-----+
| id|count|
+---+-----+
|  3|    3|
|  5|    1|
|  1|    2|
|  4|    1|
|  2|    2|
+---+-----+



In [53]:
g.vertices.join(vertexTriangles, "id", "left_outer").show()

+---+-------+---+------+----------+-----+
| id|   name|age|gender|university|count|
+---+-------+---+------+----------+-----+
|  7|   Ilya| 29|     M|       MSU| null|
|  3|Natasha| 27|     F|     SPbSU|    3|
|  5|   Oleg| 35|     M|      MIPT|    1|
|  6|   Ivan| 30|     M|       MSU| null|
|  1|   Alex| 28|     M|      MIPT|    2|
|  4|  Pavel| 30|     M|      MIPT|    1|
|  2|  Emeli| 28|     F|      MIPT|    2|
+---+-------+---+------+----------+-----+

